In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
oe=OrdinalEncoder()

In [3]:
df=pd.read_csv(r"C:\Users\vvagh\OneDrive - Indian Institute of Science Education and Research Bhopal\Documents\IITM Stuff\diploma-LittleBeasty\dataset.csv")
# df.describe()
df

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO
...,...,...,...,...,...,...
743,23.0,2.0,500.0,38.0,NEGATIVE,NO
744,21.0,2.0,500.0,52.0,NEGATIVE,NO
745,23.0,3.0,750.0,62.0,NEGATIVE,NO
746,39.0,1.0,250.0,39.0,NEGATIVE,NO


In [4]:
df.replace('?', np.nan, inplace=True)

In [5]:
df.isna().sum()

V1        5
V2        5
V3        0
V4        0
V5        0
Target    0
dtype: int64

In [6]:
df[df.isna().any(axis=1)]

,V1,V2,V3,V4,V5,Target
2,NaN,NaN,4000.0,35.0,NEGATIVE,YES
3,NaN,20.0,5000.0,45.0,NEGATIVE,YES
5,4.0,NaN,1000.0,4.0,NEGATIVE,NO
6,2.0,NaN,1750.0,14.0,NEGATIVE,YES
7,NaN,12.0,3000.0,35.0,NEGATIVE,NO
10,4.0,NaN,5750.0,58.0,NEGATIVE,NO
11,NaN,3.0,750.0,4.0,NEGATIVE,NO
16,NaN,14.0,3500.0,48.0,NEGATIVE,YES
17,2.0,NaN,3750.0,49.0,NEGATIVE,YES


In [7]:
df.dropna(inplace=True)

In [11]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression

# Encode target
y = oe.fit_transform(df[['Target']]).ravel()

# Drop the target column from X
X = df.drop('Target', axis=1)

# Define the columns for preprocessing
impute_cols = [0, 1]
scaler_cols = [0, 1, 2, 3]
onehot_cols = [4]

# Define the preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('mean_imputer', SimpleImputer(strategy='mean'), impute_cols),
        ('scaler', StandardScaler(), scaler_cols),
        ('ordinalEncoder', OrdinalEncoder(), onehot_cols)
    ]
)

# Define the pipeline with preprocessing, RFE, and LogisticRegression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('sfs', SequentialFeatureSelector(LogisticRegression(), n_features_to_select=2, direction='backward')),
    ('logreg', LogisticRegression())
])

# Fit the pipeline
pipeline.fit(X, y)

# Get the transformed features
preprocessed_X = pipeline.named_steps['preprocessor'].transform(X)
print("Shape of transformed X:", preprocessed_X.shape)

# Get selected features after RFE
selected_feature_indices = pipeline.named_steps['sfs'].get_support(indices=True)

# Print the selected features
print(f"Selected features: {selected_feature_indices}")


Shape of transformed X: (739, 7)
Selected features: [4 5]


In [12]:
selected_feature_indices

array([4, 5])

In [9]:
X.columns[selected_feature_indices]

IndexError: index 5 is out of bounds for axis 0 with size 5